In [ ]:
# Configure asyncio for Jupyter notebook
# This allows nested event loops which ib_insync needs in a notebook environment
import nest_asyncio
nest_asyncio.apply()

# Phase 3: Account Status Demo

This notebook demonstrates the account status functionality from Phase 3 of the IBKR Gateway project.

**Prerequisites:**
- IBKR Gateway or TWS running in paper trading mode
- Default connection: `127.0.0.1:4002`

## Features Demonstrated:
1. **Account summary** - Net liquidation, cash, buying power, margin
2. **Positions** - Portfolio holdings with P&L
3. **P&L** - Account-level and per-symbol profit/loss
4. **Multi-account support** - Working with multiple managed accounts
5. **Convenience functions** - Combined status retrieval
6. **Error handling** - Exception types and handling patterns

In [2]:
# Add parent directory to path for imports
import sys
sys.path.insert(0, '..')

from ibkr_core import (
    IBKRClient,
    SymbolSpec,
)

from ibkr_core.account import (
    # Core functions
    get_account_summary,
    get_positions,
    get_pnl,
    # Convenience functions
    get_account_status,
    list_managed_accounts,
    # Exceptions
    AccountError,
    AccountSummaryError,
    AccountPositionsError,
    AccountPnlError,
)

from ibkr_core.models import (
    AccountSummary,
    Position,
    AccountPnl,
    PnlDetail,
)

## 1. Connect to IBKR Gateway

In [3]:
# Create client and connect
client = IBKRClient(mode="paper")
client.connect(timeout=10)

print(f"Connected: {client.is_connected}")
print(f"Mode: {client.mode}")
print(f"Accounts: {client.managed_accounts}")

Connected: True
Mode: paper
Accounts: ['DUM096342']


Error 322, reqId 6: Error processing request.-'b0' : cause - Maximum number of account summary requests exceeded; desubscribe to previous request first
Error 322, reqId 7: Error processing request.-'b0' : cause - Maximum number of account summary requests exceeded; desubscribe to previous request first
Error 322, reqId 8: Error processing request.-'b0' : cause - Maximum number of account summary requests exceeded; desubscribe to previous request first
Error 322, reqId 9: Error processing request.-'b0' : cause - Maximum number of account summary requests exceeded; desubscribe to previous request first
Error 322, reqId 10: Error processing request.-'b0' : cause - Maximum number of account summary requests exceeded; desubscribe to previous request first
Error 322, reqId 12: Error processing request.-'b0' : cause - Maximum number of account summary requests exceeded; desubscribe to previous request first


## 2. List Managed Accounts

In multi-account environments, you may have access to multiple accounts. The `list_managed_accounts()` function returns all available account IDs.

In [4]:
# List all managed accounts
accounts = list_managed_accounts(client)

print(f"Managed Accounts ({len(accounts)}):")
for i, acc in enumerate(accounts, 1):
    print(f"  {i}. {acc}")

# We'll use the first account for the rest of the demo
default_account = accounts[0] if accounts else None
print(f"\nUsing account: {default_account}")

Managed Accounts (1):
  1. DUM096342

Using account: DUM096342


## 3. Account Summary

Get a snapshot of account balances, margin requirements, and buying power.

In [5]:
# Get account summary (uses default account if account_id not specified)
summary = get_account_summary(client)

print("Account Summary")
print("=" * 50)
print(f"Account ID:          {summary.accountId}")
print(f"Currency:            {summary.currency}")
print(f"Net Liquidation:     ${summary.netLiquidation:,.2f}")
print(f"Cash Balance:        ${summary.cash:,.2f}")
print(f"Buying Power:        ${summary.buyingPower:,.2f}")
print(f"Margin Excess:       ${summary.marginExcess:,.2f}")
print(f"Maintenance Margin:  ${summary.maintenanceMargin:,.2f}")
print(f"Initial Margin:      ${summary.initialMargin:,.2f}")
print(f"Timestamp:           {summary.timestamp}")

Account Summary
Account ID:          DUM096342
Currency:            CAD
Net Liquidation:     $1,011,933.96
Cash Balance:        $1,010,551.18
Buying Power:        $3,372,285.66
Margin Excess:       $1,011,708.27
Maintenance Margin:  $225.69
Initial Margin:      $248.26
Timestamp:           2025-12-17 02:51:15.711820+00:00


In [6]:
# Get account summary for a specific account (explicit account_id)
if len(accounts) > 0:
    specific_summary = get_account_summary(client, account_id=accounts[0])
    print(f"Summary for {specific_summary.accountId}: NLV = ${specific_summary.netLiquidation:,.2f}")

Summary for DUM096342: NLV = $1,011,933.96


## 4. Portfolio Positions

Retrieve all open positions with market values and P&L.

In [7]:
# Get all positions
positions = get_positions(client)

print(f"Portfolio Positions ({len(positions)} total)")
print("=" * 100)

if positions:
    print(f"{'Symbol':<15} {'Class':<6} {'Qty':>10} {'Avg Price':>12} {'Mkt Price':>12} {'Mkt Value':>14} {'Unreal P&L':>12}")
    print("-" * 100)
    for pos in positions:
        print(f"{pos.symbol:<15} {pos.assetClass:<6} {pos.quantity:>10.2f} {pos.avgPrice:>12.2f} {pos.marketPrice:>12.2f} {pos.marketValue:>14.2f} {pos.unrealizedPnl:>12.2f}")
    
    # Calculate totals
    total_value = sum(pos.marketValue for pos in positions)
    total_unrealized = sum(pos.unrealizedPnl for pos in positions)
    print("-" * 100)
    print(f"{'TOTAL':<15} {'':<6} {'':<10} {'':<12} {'':<12} {total_value:>14.2f} {total_unrealized:>12.2f}")
else:
    print("No open positions")

Portfolio Positions (1 total)
Symbol          Class         Qty    Avg Price    Mkt Price      Mkt Value   Unreal P&L
----------------------------------------------------------------------------------------------------
AAPL            STK          2.00       101.06       273.20         546.40       142.16
----------------------------------------------------------------------------------------------------
TOTAL                                                               546.40       142.16


In [8]:
# Inspect a single position in detail
if positions:
    pos = positions[0]
    print(f"Position Details: {pos.symbol}")
    print("=" * 40)
    print(f"Account ID:      {pos.accountId}")
    print(f"Symbol:          {pos.symbol}")
    print(f"Contract ID:     {pos.conId}")
    print(f"Asset Class:     {pos.assetClass}")
    print(f"Currency:        {pos.currency}")
    print(f"Quantity:        {pos.quantity}")
    print(f"Avg Price:       ${pos.avgPrice:.2f}")
    print(f"Market Price:    ${pos.marketPrice:.2f}")
    print(f"Market Value:    ${pos.marketValue:.2f}")
    print(f"Unrealized P&L:  ${pos.unrealizedPnl:.2f}")
    print(f"Realized P&L:    ${pos.realizedPnl:.2f}")

Position Details: AAPL
Account ID:      DUM096342
Symbol:          AAPL
Contract ID:     265598
Asset Class:     STK
Currency:        USD
Quantity:        2.0
Avg Price:       $101.06
Market Price:    $273.20
Market Value:    $546.40
Unrealized P&L:  $142.16
Realized P&L:    $0.00


## 5. Profit & Loss (P&L)

Get account-level and per-symbol P&L breakdown.

**Note:** The `timeframe` parameter is reserved for future use. Currently only returns current P&L.

In [9]:
# Get account P&L
pnl = get_pnl(client)

print("Account P&L Summary")
print("=" * 50)
print(f"Account ID:    {pnl.accountId}")
print(f"Currency:      {pnl.currency}")
print(f"Timeframe:     {pnl.timeframe}")
print(f"Realized:      ${pnl.realized:,.2f}")
print(f"Unrealized:    ${pnl.unrealized:,.2f}")
print(f"Total P&L:     ${(pnl.realized + pnl.unrealized):,.2f}")
print(f"Timestamp:     {pnl.timestamp}")

Account P&L Summary
Account ID:    DUM096342
Currency:      CAD
Timeframe:     CURRENT
Realized:      $0.00
Unrealized:    $142.16
Total P&L:     $142.16
Timestamp:     2025-12-17 02:52:09.067286+00:00


In [10]:
# Per-symbol P&L breakdown
print(f"\nP&L by Symbol ({len(pnl.bySymbol)} symbols)")
print("=" * 60)

if pnl.bySymbol:
    print(f"{'Symbol':<15} {'Realized':>15} {'Unrealized':>15} {'Total':>15}")
    print("-" * 60)
    for symbol, detail in pnl.bySymbol.items():
        total = detail.realized + detail.unrealized
        print(f"{symbol:<15} ${detail.realized:>13,.2f} ${detail.unrealized:>13,.2f} ${total:>13,.2f}")
else:
    print("No P&L data (no open positions)")


P&L by Symbol (1 symbols)
Symbol                 Realized      Unrealized           Total
------------------------------------------------------------
AAPL            $         0.00 $       142.16 $       142.16


## 6. Combined Account Status

The `get_account_status()` convenience function retrieves both summary and positions in a single call.

In [11]:
# Get combined account status
status = get_account_status(client)

print("Combined Account Status")
print("=" * 50)
print(f"\nAccount: {status['summary'].accountId}")
print(f"Net Liquidation: ${status['summary'].netLiquidation:,.2f}")
print(f"Open Positions: {len(status['positions'])}")

if status['positions']:
    print("\nTop positions by market value:")
    sorted_positions = sorted(status['positions'], key=lambda p: abs(p.marketValue), reverse=True)
    for pos in sorted_positions[:5]:
        print(f"  {pos.symbol}: ${pos.marketValue:,.2f} (P&L: ${pos.unrealizedPnl:,.2f})")

Combined Account Status

Account: DUM096342
Net Liquidation: $1,011,933.96
Open Positions: 1

Top positions by market value:
  AAPL: $546.40 (P&L: $142.16)


## 7. Multi-Account Support

All account functions accept an optional `account_id` parameter for multi-account environments.

In [12]:
# Demonstrate multi-account support
accounts = list_managed_accounts(client)

if len(accounts) >= 1:
    print("Account Summary Comparison")
    print("=" * 70)
    print(f"{'Account':<15} {'NLV':>15} {'Cash':>15} {'Buying Power':>15}")
    print("-" * 70)
    
    for acc_id in accounts:
        try:
            acc_summary = get_account_summary(client, account_id=acc_id)
            print(f"{acc_id:<15} ${acc_summary.netLiquidation:>13,.2f} ${acc_summary.cash:>13,.2f} ${acc_summary.buyingPower:>13,.2f}")
        except AccountSummaryError as e:
            print(f"{acc_id:<15} Error: {e}")
else:
    print("Only one account available")

Account Summary Comparison
Account                     NLV            Cash    Buying Power
----------------------------------------------------------------------
DUM096342       $ 1,011,933.96 $ 1,010,551.18 $ 3,372,285.66


## 8. Error Handling

The account module provides specific exception types for different error conditions.

In [13]:
# Exception hierarchy
print("Exception Hierarchy:")
print("  AccountError (base)")
print("    ├── AccountSummaryError")
print("    ├── AccountPositionsError")
print("    └── AccountPnlError")

# Verify inheritance
print(f"\nAccountSummaryError inherits from AccountError: {issubclass(AccountSummaryError, AccountError)}")
print(f"AccountPositionsError inherits from AccountError: {issubclass(AccountPositionsError, AccountError)}")
print(f"AccountPnlError inherits from AccountError: {issubclass(AccountPnlError, AccountError)}")

Exception Hierarchy:
  AccountError (base)
    ├── AccountSummaryError
    ├── AccountPositionsError
    └── AccountPnlError

AccountSummaryError inherits from AccountError: True
AccountPositionsError inherits from AccountError: True
AccountPnlError inherits from AccountError: True


In [14]:
# Handle invalid account ID
try:
    invalid_summary = get_account_summary(client, account_id="INVALID_ACCOUNT_12345")
except AccountSummaryError as e:
    print(f"AccountSummaryError: {e}")
except AccountError as e:
    print(f"AccountError: {e}")

AccountSummaryError: No account summary values found for account INVALID_ACCOUNT_12345


## 9. Model Inspection

Explore the Pydantic models used for account data.

In [15]:
# Inspect AccountSummary model fields
print("AccountSummary Model Fields:")
for field_name, field_info in AccountSummary.model_fields.items():
    print(f"  {field_name}: {field_info.annotation}")

AccountSummary Model Fields:
  accountId: <class 'str'>
  currency: <class 'str'>
  netLiquidation: <class 'float'>
  cash: <class 'float'>
  buyingPower: <class 'float'>
  marginExcess: <class 'float'>
  maintenanceMargin: <class 'float'>
  initialMargin: <class 'float'>
  timestamp: <class 'datetime.datetime'>


In [16]:
# Inspect Position model fields
print("Position Model Fields:")
for field_name, field_info in Position.model_fields.items():
    print(f"  {field_name}: {field_info.annotation}")

Position Model Fields:
  accountId: <class 'str'>
  symbol: <class 'str'>
  conId: <class 'int'>
  assetClass: <class 'str'>
  currency: <class 'str'>
  quantity: <class 'float'>
  avgPrice: <class 'float'>
  marketPrice: <class 'float'>
  marketValue: <class 'float'>
  unrealizedPnl: <class 'float'>
  realizedPnl: <class 'float'>


In [17]:
# Inspect AccountPnl model fields
print("AccountPnl Model Fields:")
for field_name, field_info in AccountPnl.model_fields.items():
    print(f"  {field_name}: {field_info.annotation}")

AccountPnl Model Fields:
  accountId: <class 'str'>
  currency: <class 'str'>
  timeframe: <class 'str'>
  realized: <class 'float'>
  unrealized: <class 'float'>
  bySymbol: typing.Dict[str, ibkr_core.models.PnlDetail]
  timestamp: <class 'datetime.datetime'>


In [18]:
# JSON serialization example
print("AccountSummary as JSON:")
print(summary.model_dump_json(indent=2))

AccountSummary as JSON:
{
  "accountId": "DUM096342",
  "currency": "CAD",
  "netLiquidation": 1011933.96,
  "cash": 1010551.18,
  "buyingPower": 3372285.66,
  "marginExcess": 1011708.27,
  "maintenanceMargin": 225.69,
  "initialMargin": 248.26,
  "timestamp": "2025-12-17T02:51:15.711820Z"
}


In [19]:
# Position as JSON (if available)
if positions:
    print("Position as JSON:")
    print(positions[0].model_dump_json(indent=2))

Position as JSON:
{
  "accountId": "DUM096342",
  "symbol": "AAPL",
  "conId": 265598,
  "assetClass": "STK",
  "currency": "USD",
  "quantity": 2.0,
  "avgPrice": 101.060025,
  "marketPrice": 273.2000122,
  "marketValue": 546.4,
  "unrealizedPnl": 142.16,
  "realizedPnl": 0.0
}


## 10. Practical Example: Portfolio Dashboard

Combine all account functions to create a simple portfolio dashboard.

In [20]:
def print_portfolio_dashboard(client):
    """Print a simple portfolio dashboard."""
    # Get all data
    summary = get_account_summary(client)
    positions = get_positions(client)
    pnl = get_pnl(client)
    
    # Header
    print("\n" + "=" * 70)
    print(f"  PORTFOLIO DASHBOARD - {summary.accountId}")
    print(f"  {summary.timestamp.strftime('%Y-%m-%d %H:%M:%S')} UTC")
    print("=" * 70)
    
    # Account Overview
    print("\n📊 ACCOUNT OVERVIEW")
    print("-" * 40)
    print(f"  Net Liquidation:    ${summary.netLiquidation:>15,.2f}")
    print(f"  Cash Balance:       ${summary.cash:>15,.2f}")
    print(f"  Buying Power:       ${summary.buyingPower:>15,.2f}")
    print(f"  Margin Available:   ${summary.marginExcess:>15,.2f}")
    
    # P&L Summary
    print("\n💰 P&L SUMMARY")
    print("-" * 40)
    print(f"  Unrealized P&L:     ${pnl.unrealized:>15,.2f}")
    print(f"  Realized P&L:       ${pnl.realized:>15,.2f}")
    total_pnl = pnl.realized + pnl.unrealized
    pnl_color = "🟢" if total_pnl >= 0 else "🔴"
    print(f"  Total P&L:          ${total_pnl:>15,.2f} {pnl_color}")
    
    # Positions
    print(f"\n📈 POSITIONS ({len(positions)})")
    print("-" * 70)
    if positions:
        # Sort by absolute market value
        sorted_pos = sorted(positions, key=lambda p: abs(p.marketValue), reverse=True)
        print(f"  {'Symbol':<12} {'Qty':>8} {'Price':>10} {'Value':>12} {'P&L':>10}")
        print("  " + "-" * 56)
        for pos in sorted_pos:
            pnl_indicator = "🟢" if pos.unrealizedPnl >= 0 else "🔴"
            print(f"  {pos.symbol:<12} {pos.quantity:>8.0f} ${pos.marketPrice:>8.2f} ${pos.marketValue:>10,.2f} ${pos.unrealizedPnl:>8,.2f} {pnl_indicator}")
    else:
        print("  No open positions")
    
    print("\n" + "=" * 70)

# Run the dashboard
print_portfolio_dashboard(client)


  PORTFOLIO DASHBOARD - DUM096342
  2025-12-17 02:52:57 UTC

📊 ACCOUNT OVERVIEW
----------------------------------------
  Net Liquidation:    $   1,011,933.97
  Cash Balance:       $   1,010,551.13
  Buying Power:       $   3,372,285.65
  Margin Available:   $   1,011,708.27

💰 P&L SUMMARY
----------------------------------------
  Unrealized P&L:     $         142.16
  Realized P&L:       $           0.00
  Total P&L:          $         142.16 🟢

📈 POSITIONS (1)
----------------------------------------------------------------------
  Symbol            Qty      Price        Value        P&L
  --------------------------------------------------------
  AAPL                2 $  273.20 $    546.40 $  142.16 🟢



## 11. Disconnect

In [21]:
# Clean up - disconnect from gateway
client.disconnect()
print(f"Disconnected. Is connected: {client.is_connected}")

Disconnected. Is connected: False


## Summary

Phase 3 provides comprehensive account status functionality with multi-account support.

### Core Functions:
- `get_account_summary(client, account_id=None)` - Account balances and margin
- `get_positions(client, account_id=None)` - Portfolio holdings with P&L
- `get_pnl(client, account_id=None, timeframe=None)` - P&L breakdown by symbol

### Convenience Functions:
- `get_account_status(client, account_id=None)` - Combined summary + positions
- `list_managed_accounts(client)` - List all available accounts

### Data Models:
- `AccountSummary` - NLV, cash, buying power, margin requirements
- `Position` - Symbol, quantity, prices, market value, P&L
- `AccountPnl` - Account-level and per-symbol P&L
- `PnlDetail` - Per-symbol realized/unrealized P&L

### Exception Classes:
- `AccountError` - Base exception for account operations
- `AccountSummaryError` - Account summary retrieval failed
- `AccountPositionsError` - Positions retrieval failed
- `AccountPnlError` - P&L retrieval failed

### Key Features:
- Multi-account support via `account_id` parameter
- JSON-serializable Pydantic models
- Futures positions include expiry in symbol (e.g., `MES_20251219`)
- P&L aggregates by base symbol for derivatives
- Clean exception hierarchy for error handling